# Individual Project 1
# Meitong Chen

### Part 1: Scraping and Saving HTML Content

In [46]:
 # (1) Identify the Target:

 # (2) Interact with the Page-Sorting:
       #  Q: Observe any changes in the URL after you change the sorting order back and forth:
       #  A:  For the newest: URL=https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0
       #      For the oldest: URL=https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=send data to a server to create/update a resource.1~gallery~0~0
       #      The "sort=date" means that the newest first, "sort=dateoldest" means that the oldest first. Other parts did not change.
       #  Q: Can you trigger the sorting change directly by modifying only the URL in your browser’s address bar?  If so, how?
       #  A: Yes, I can. Change the "sort=date" into "sort=dateoldest", other parts still same.
       #  Q: Explain what type of request is made when you change the sort order (GET or POST).
       #  A: I used the GET request. Because I only need to request data from website, but do not need to send data to a server to create or update.
       #  Q: What is the variable in the URL associated with sorting?
       #  A: The sort variable.

# (3) Interact with the Page-Pagination:
       #  Q: Determine how to move between pages by only changing the URL.  What part of the URL changes as you navigate through different pages?
       #  A: For the first page: https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0
       #     For the second page: https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~1~0
       #     For the third page: https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~2~0
       #     And for the third page, when I slided the mouse down, it changed to be: https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~2~6(this last number changed)
       #  Q: Identify the variable associated with page changes.  How does altering this variable in the URL affect the page you’re viewing?
       #  A: The first number after "~gallery~" represents the page number or a set of listings being viewed. For example: 0 for the first page, 1 for the second page,2 for the third page
       #     (additional: The second number after "~gallery~" means which part to view. The first part is 0, second part is 1,etc.
        
# (4) Fetch Listing URLs:
import requests
from bs4 import BeautifulSoup
import time

# same part of url for diff pages
samepart_url = 'https://sfbay.craigslist.org/search/zip'

# to be honest, for one page can get 120, 250/120<3
for gallery_number in range(3):  
    url = f"{samepart_url}?sort=date#search=1~gallery~{gallery_number*120}"  # for previous questions, i knew that the first number after gallery is the page number
    headers = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # use css selectors 
    # ol is the direct parent of li
    listing = soup.select('ol > li')
    
    # second method:
    listing = soup.find_all('li')
    
    # set a gap time between 2 requests
    time.sleep(5)
    # set an empty list to store later
    links = []
    # because the a is not the direct son of li, i used find here
    for n in listing:
        a = n.find('a', href=True)
        if a:
            href = a['href']
            if href not in links:
                links.append(href) # make sure the links have not appeared before
        if len(links) == 250:  # we only need 250
            break
    if len(links) == 250:
        break

for url in links:
    print(url)
print(len(links))

       #  Q: What selector do you choose to grab the link?  Explain your strategy.
       #  A: A css selector by using selector of beautifulsoup, with the relationship between ol and li. The ol is the direct parent of li, and a is inside of li.
       #  Q: Can you identify one more possible selection method to retrieve the link to the individual listing?  Explain.
       #  A: Or directly use the li selector, and find a['href'] inside of li.

# (5) Save HTML Pages:
import requests
import os
import time
from urllib.parse import urlparse

# i wanna download them on desktop in a folder craigslist_listings
save_path = os.path.join('Desktop', 'craigslist_listings')
os.makedirs(save_path, exist_ok=True)

# input the links
links_list = links  

# one html for one doc
for url in links_list[:250]:  
    listing_id = urlparse(url).path.split('/')[-1].split('.')[0]
    filename = os.path.join(save_path, f"{listing_id}.html")
    response = requests.get(url) # get the request as needed
    if response.status_code == 200:
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(response.text)
    print(f"Saved {url} to {filename}")
    

    time.sleep(5)

        

https://sfbay.craigslist.org/eby/zip/d/san-leandro-offset-printing/7709342694.html
https://sfbay.craigslist.org/eby/zip/d/pittsburg-free-gas-stove/7715584866.html
https://sfbay.craigslist.org/eby/zip/d/alameda-queen-sized-metal-bed-frame-no/7715584857.html
https://sfbay.craigslist.org/nby/zip/d/sebastopol-couple-hanging-lamps/7710179092.html
https://sfbay.craigslist.org/sfc/zip/d/san-francisco-rug-and-file-cabinet/7715583990.html
https://sfbay.craigslist.org/eby/zip/d/antioch-bookcases-flags-hats-more/7715583130.html
https://sfbay.craigslist.org/sby/zip/d/los-gatos-desk-free/7715582939.html
https://sfbay.craigslist.org/eby/zip/d/hayward-foam-container-large/7708222487.html
https://sfbay.craigslist.org/pen/zip/d/palo-alto-free-laundry/7715581951.html
https://sfbay.craigslist.org/sby/zip/d/cupertino-dining-chair-for-free/7714182938.html
https://sfbay.craigslist.org/sfc/zip/d/san-francisco-shelves-cabinets/7715580951.html
https://sfbay.craigslist.org/sby/zip/d/san-jose-shiny-metal-pc-bath

### Part 2: Parsing and Displaying Information from Saved HTML

In [49]:
import os
from bs4 import BeautifulSoup

# (1) Read Saved HTML Files:
# path
path = os.path.join('Desktop', 'craigslist_listings')

# go through each one
for filename in os.listdir(path):
    filepath = os.path.join(path, filename)
    with open(filepath, 'r', encoding='utf-8') as file:
            html_content = file.read()
            
# (2) Extract Information:
# use beautiful to read
for filename in os.listdir(path):
    filepath = os.path.join(path, filename)
    with open(filepath, 'r', encoding='utf-8') as file:
            soup = BeautifulSoup(file.read(), 'html.parser')
            
            # title 
            title = soup.find('title').get_text() if soup.find('title') else 'None Title'
            
            # image
            image = soup.find('img')
            image_url = image['src'] if image and'src' in image.attrs else 'None Image'
            
            # select the description <section id="postingbody">
            description_selector = soup.find('section', id='postingbody')
            description = description_selector.get_text(strip=True) if description_selector else 'None Description'
            
            # post id by p is the son of div, which class_='postinginfos'
            post_div = soup.find('div', class_='postinginfos')
            post_p = post_div.find('p', class_='postinginfo') if post_div else None
            post_id = post_p.get_text().split(':')[-1].strip() if post_p else 'None Post ID'

            # posted date and last updated date under p with class_='postinginfo reveal',and time is the son of p
            posting_info_tags = soup.find_all('p', class_='postinginfo reveal')
            posted_date = 'None Posted Date'
            last_updated_date = 'None Last Updated Date'
            for tag in posting_info_tags:
                if 'posted:' in tag.text:
                    posted_date = tag.find('time')['datetime']
                elif 'updated:' in tag.text:
                    last_updated_date = tag.find('time')['datetime']

            print(f"Filename: {filename}")
            print(f"Title: {title}")
            print(f"URL of first image: {image_url}")
            print(f"Description: {description}")
            print(f"Post ID: {post_id}")
            print(f"Posted Date: {posted_date}")
            print(f"Last Updated Date: {last_updated_date}")
            print("\n")


Filename: 7705769410.html
Title: Nursing school CD’s - free stuff - craigslist
URL of first image: https://images.craigslist.org/00g0g_xqpqng2aoy_0lM0CI_600x450.jpg
Description: QR Code Link to This PostThese CD’s are from 2005-2007. They came with my skill books and textbooks. I don’t know if anyone uses CD’s anymore?
Post ID: 7705769410
Posted Date: 2024-01-09T08:39:12-0800
Last Updated Date: 2024-02-07T17:05:13-0800


Filename: 7706154406.html
Title: JOVITEC 12 Pieces Rubber Bracelets, Solid Color Silicone Wristbands - free stuff - craigslist
URL of first image: https://images.craigslist.org/00505_fYDvpuejzeo_0oc0nn_600x450.jpg
Description: QR Code Link to This PostItem: JOVITEC 12 Pieces Rubber Bracelets, Solid Color Silicone Wristbands, Multi-Pack Blank Wristbands Bracelets for Events Rubber Bands Party (Pink, Blue, Yellow)This is a new pack of JOVITEC 12 Pieces Rubber Bracelets, Solid Color Silicone Wristbands.  I bought this brand new in December 2023 and I currently don't have 

### Part 3: Automating Login on The Old Reader

In [50]:
# (1) Creating and Verifying a The Old Reader Account:
# (2) Exploring the Login Mechanism:
from bs4 import BeautifulSoup
import requests
import time

headers = {'User-agent': 'Mozilla/5.0'}

url = 'https://theoldreader.com/users/sign_in'
page = requests.get(url, headers=headers) 
soup = BeautifulSoup(page.content, 'html.parser')

form = soup.find('form')

# output the form, thw value and the name
if form:
    input1 = form.find_all('input')
    for field in input1:
        field_name = field.get('name')
        field_value = field.get('value', '')
        print({field_name}, {field_value})


# (3) Analyzing Network Traffic for Login Request:

       #  Q: Why POST for Login Forms?
       #  A: Post  is used to submit data to the resource, will have effect on server.When the it is related to login, i think it should be post request.
       #  Q: Examining the Payload
       #  A: At first: in <form accept-charset="UTF-8" action="/users/sign_in" class="new_user" id="new_user" method="post" role="form">
       #                  <div style="margin:0;padding:0;display:inline">
       #                  <input name="utf8" type="hidden" value="✓"/>
       #                  <input name="authenticity_token" type="hidden" value="fttkrvo43/kqThD9dsY4ZGKkaW3UzyR8XiC9dUVmrjQ="/>
       #                 </div>
       #                 <div class="form-group">
       #                  <input autocapitalize="off" autocorrect="off" autofocus="autofocus" class="form-control" id="user_login" name="user[login]" placeholder="Username/Email" size="30" spellcheck="false" type="text"/>
       #                 </div>
       #                <div class="form-group">
       #                <input class="form-control" id="user_password" name="user[password]" placeholder="Password" size="30" type="password"/>
       #                 </div>
       #                 <input class="btn btn-primary btn-block" name="commit" type="submit" value="Sign In"/>
       #                  </form> 
       #                 when i login, they are not able to see
    

# (4) Automating the Login Process:
import requests
from bs4 import BeautifulSoup

# login page
url = 'https://theoldreader.com/users/sign_in'

# session for cookies
session = requests.Session()
res = session.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

# output some nonstatic and hard to print values,
utf8_value = soup.select_one('input[name=utf8]')['value']
authenticity_token = soup.find('input',{'name':'authenticity_token'})['value']

# requset inside our session information
payload = {
    'utf8': utf8_value,
    'authenticity_token': authenticity_token,
    'user[login]': 'pdmeitongchen@126.com', 
    'user[password]': '12345678',           
    'commit': 'Sign In'
}


login_response = session.post(url, data=payload)

# get the cookies for login
cookies = session.cookies.get_dict()
print(cookies)


# (5) Automating the Login Process:
import requests

# function for the text
def function_text(response, text):
    return text in response.text

# session for cookies
session = requests.Session()
res = session.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

# output some nonstatic and hard to print values,
utf8_value = soup.select_one('input[name=utf8]')['value']
authenticity_token = soup.find('input',{'name':'authenticity_token'})['value']

# requset inside our session information
payload = {
    'utf8': utf8_value,
    'authenticity_token': authenticity_token,
    'user[login]': 'pdmeitongchen@126.com', 
    'user[password]': '12345678',           
    'commit': 'Sign In'
}

# login
response = session.post(url, data=credentials)

# use the page which after login to check the words only for login, i used my name'Chen'
loggedin_url = 'https://theoldreader.com/'
response = session.get(loggedin_url)
if contains_text(response, 'Chen'):
    print('Accessed protected page successfully.')
else:
    print('Failed to access protected page.')



{'utf8'} {'✓'}
{'authenticity_token'} {'l2T9DaZCq1M26kHMhWa+S+pc9oanIbwaoBON2aFrjbg='}
{'user[login]'} {''}
{'user[password]'} {''}
{'commit'} {'Sign In'}
{'_new_reader_session': 'BAh7CkkiD3Nlc3Npb25faWQGOgZFVEkiJTEyYjM3MGJmMzE0YzEyNzU4NDVkNTRmZjgyZTJkZmU4BjsAVEkiDWxhbmd1YWdlBjsARjoHZW5JIhByZWRpcmVjdF90bwY7AEZJIgYvBjsARkkiGXdhcmRlbi51c2VyLnVzZXIua2V5BjsAVFsHWwZVOhpNb3BlZDo6QlNPTjo6T2JqZWN0SWQiERz817Eh8NnyCI4gd0kiIiQyYSQwNSRvVkdCMDByVy92dDFXbGVKZzJJeFd1BjsAVEkiEF9jc3JmX3Rva2VuBjsARkkiMU5RNExrYU1UejZvU1o0eTNJZDhQalVPdEVkbFFhVjlCNG1ER3k2NU1DSnc9BjsARg%3D%3D--c60afe321e32f042aab736c4fa9b95ee8ace012a', 'i_know_you': 'Chen', 'remember_user_token': 'BAhbB1sGVToaTW9wZWQ6OkJTT046Ok9iamVjdElkIhEc%2FNexIfDZ8giOIHdJIiIkMmEkMDUkb1ZHQjAwclcvdnQxV2xlSmcySXhXdQY6BkVU--931fe0a31d7d324d28cb3713e1b4bd28330b5279', 'signed_at': '1707379095'}
Accessed protected page successfully.
